# 03. 결과 분석 및 시각화

모델 예측 결과를 분석하고 최종 입지 추천을 시각화합니다.

## 주요 내용
- 상위/하위 지역 비교
- 피처별 영향도 분석
- 최종 입지 추천

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append('..')

from src.data_loader import load_csv
from src.visualization import ResultVisualizer, set_korean_font

# 한글 폰트 설정 (OS 자동 감지)
set_korean_font()

%matplotlib inline

## 1. 결과 데이터 로드

In [ ]:
# 데이터 로드
train_df = load_csv('../data/processed/train_dataset.csv')
test_df = load_csv('../data/processed/test_dataset.csv')

# 컬럼명 통일
if '30~59세 인구' in train_df.columns:
    train_df = train_df.rename(columns={'30~59세 인구': '30~59세'})

feature_cols = ['평균 버스 이용량 (명)', '1~2인가구', '3인 가구 이상', '30~59세', '비만도 분포', '가구', '수요', '인프라', '경제']

print(f'Train: {len(train_df)}개 샘플')
print(f'Test: {len(test_df)}개 행정동')
print(f'피처: {len(feature_cols)}개')

## 2. 상위/하위 지역 분석

In [ ]:
# 모델 학습 및 예측
from sklearn.ensemble import RandomForestClassifier

X_train = train_df[feature_cols]
y_train = train_df['target']
X_test = test_df[feature_cols]

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# 예측 점수 (입지 적합 확률)
test_df['pred_score'] = rf.predict_proba(X_test)[:, 1]

# 상위/하위 15개 지역
top15 = test_df.nlargest(15, 'pred_score')
bottom15 = test_df.nsmallest(15, 'pred_score')

print('=== 상위 15개 추천 지역 ===')
for i, (_, row) in enumerate(top15.iterrows(), 1):
    print(f'{i:2}. {row["구_x"]} {row["행정동_x"]} (점수: {row["pred_score"]:.2f})')

In [ ]:
# 상위/하위 비교 시각화
viz = ResultVisualizer(figsize=(12, 6))
viz.set_output_dir('../outputs/figures')

top_idx = test_df.nlargest(15, 'pred_score').index
bottom_idx = test_df.nsmallest(15, 'pred_score').index

top_mean = test_df.loc[top_idx, feature_cols].mean()
bottom_mean = test_df.loc[bottom_idx, feature_cols].mean()

# 비교 데이터프레임 생성
comparison_df = pd.DataFrame({
    '피처': feature_cols,
    '상위 15개': top_mean.values,
    '하위 15개': bottom_mean.values,
    '차이': (top_mean - bottom_mean).values
}).round(3)

print(comparison_df.to_string(index=False))

# 시각화
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(feature_cols))
width = 0.35

ax.bar(x - width/2, top_mean, width, label='상위 15개 지역', color='#2ecc71')
ax.bar(x + width/2, bottom_mean, width, label='하위 15개 지역', color='#e74c3c')

ax.set_ylabel('평균값 (정규화)')
ax.set_title('상위 vs 하위 지역 피처 비교')
ax.set_xticks(x)
ax.set_xticklabels(feature_cols, rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.savefig('../outputs/figures/district_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. 예측 점수 분포

In [ ]:
# 상위 15개 지역 시각화
fig, ax = plt.subplots(figsize=(10, 8))

top15_sorted = top15.sort_values('pred_score')
colors = plt.cm.RdYlGn(np.linspace(0.3, 0.9, 15))

labels = [f"{row['구_x']} {row['행정동_x']}" for _, row in top15_sorted.iterrows()]
bars = ax.barh(range(15), top15_sorted['pred_score'], color=colors)

ax.set_yticks(range(15))
ax.set_yticklabels(labels)
ax.set_xlabel('예측 점수 (입지 적합 확률)')
ax.set_title('밀키트 매장 입지 추천 상위 15개 지역')
ax.set_xlim(0, 1)
ax.grid(axis='x', alpha=0.3)

for bar, score in zip(bars, top15_sorted['pred_score']):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height()/2, 
            f'{score:.2f}', va='center', fontsize=10)

plt.tight_layout()
plt.savefig('../outputs/figures/top_districts.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. 결론

In [ ]:
print('='*60)
print('서울시 밀키트 매장 최적 입지 분석 결과')
print('='*60)
print()
print('1. 상위 추천 지역 (Top 5):')
for i, (_, row) in enumerate(top15.head(5).iterrows(), 1):
    print(f'   {i}. {row["구_x"]} {row["행정동_x"]} (점수: {row["pred_score"]:.2f})')
print()
print('2. 피처 중요도 (Random Forest):')
importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)
for _, row in importance.head(5).iterrows():
    print(f'   - {row["feature"]}: {row["importance"]:.3f}')
print()
print('3. 상위/하위 지역 특성:')
print('   - 상위 지역: 인프라/경제 지표가 낮은 주거 밀집 지역')
print('   - 하위 지역: 인프라/경제 지표가 높은 상업 중심 지역')
print()
print('4. 한계점:')
print('   - 실제 매출 데이터 없이 프록시 변수 사용')
print('   - 표본 크기 제한 (183개 매장)으로 일반화 주의 필요')